In [ ]:
!pip install tokenizers
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [ ]:
!unzip "data 1.zip" -d "data"

Archive:  data 1.zip
  inflating: data/data 1/00211-027889-DISCHARGE_SUMMARY.txt  
  inflating: data/data 1/00414-104513-ECHO_REPORT.txt  
  inflating: data/data 1/00500-097836-ECHO_REPORT.txt  
  inflating: data/data 1/01114-083601-ECG_REPORT.txt  
  inflating: data/data 1/01234-029456-DISCHARGE_SUMMARY.txt  
  inflating: data/data 1/01455-067052-ECG_REPORT.txt  
  inflating: data/data 1/01487-290421-RADIOLOGY_REPORT.txt  
  inflating: data/data 1/01982-060190-ECG_REPORT.txt  
  inflating: data/data 1/02034-037300-ECG_REPORT.txt  
  inflating: data/data 1/02115-010823-DISCHARGE_SUMMARY.txt  
  inflating: data/data 1/02136-017465-DISCHARGE_SUMMARY.txt  
  inflating: data/data 1/02405-069810-ECG_REPORT.txt  
  inflating: data/data 1/02410-026171-DISCHARGE_SUMMARY.txt  
  inflating: data/data 1/02652-006395-DISCHARGE_SUMMARY.txt  
  inflating: data/data 1/02916-100844-ECHO_REPORT.txt  
  inflating: data/data 1/03066-084521-ECG_REPORT.txt  
  inflating: data/data 1/03089-097913-ECHO_REPOR

In [ ]:
import pandas as pd
import os
import glob
from tokenizers import ByteLevelBPETokenizer
import torch
from transformers import AdamW
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm.auto import tqdm
import numpy as np
import os
import re
from transformers import BertTokenizer, BertForMaskedLM, BertConfig
from datasets import Dataset
import numpy as np
from tokenizers import BertWordPieceTokenizer

In [ ]:
NUM_EPOCHS = 1


In [ ]:
# store path to the text file
text_file_paths = glob.glob("/content/data/data 1/*.txt")

In [ ]:
text_file_paths

['/content/data/data 1/26563-387055-RADIOLOGY_REPORT.txt',
 '/content/data/data 1/18673-102519-ECHO_REPORT.txt',
 '/content/data/data 1/22821-026994-DISCHARGE_SUMMARY.txt',
 '/content/data/data 1/24432-013472-DISCHARGE_SUMMARY.txt',
 '/content/data/data 1/09375-099229-ECHO_REPORT.txt',
 '/content/data/data 1/14158-075452-ECG_REPORT.txt',
 '/content/data/data 1/19623-085193-ECG_REPORT.txt',
 '/content/data/data 1/02115-010823-DISCHARGE_SUMMARY.txt',
 '/content/data/data 1/17522-024788-DISCHARGE_SUMMARY.txt',
 '/content/data/data 1/06445-096221-ECHO_REPORT.txt',
 '/content/data/data 1/12748-021750-DISCHARGE_SUMMARY.txt',
 '/content/data/data 1/09584-107853-ECHO_REPORT.txt',
 '/content/data/data 1/08014-097374-ECHO_REPORT.txt',
 '/content/data/data 1/07780-347384-RADIOLOGY_REPORT.txt',
 '/content/data/data 1/04269-027967-DISCHARGE_SUMMARY.txt',
 '/content/data/data 1/16660-199016-RADIOLOGY_REPORT.txt',
 '/content/data/data 1/16994-022078-DISCHARGE_SUMMARY.txt',
 '/content/data/data 1/0656

In [ ]:
def preprocess_text(text):
    text = re.sub(r'\n+', ' ', text)  # Replace newlines with space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'\d+', '[NUM]', text)  # Replace digits with a placeholder
    # text = re.sub(r'\[\*\*(.*?)\*\*\]', '[REDACTED]', text)  # Replace redacted parts
    return text

In [ ]:
# Read and preprocess all text files
all_texts = []
for filename in text_file_paths:
    print(filename)
    with open(filename, 'r', encoding='utf-8') as file:

        raw_text = file.read()
        print(raw_text)
        preprocessed_text = preprocess_text(raw_text)
        print(preprocessed_text)
        all_texts.append(preprocessed_text)

# Save preprocessed text to a temporary file for tokenizer training
with open('combined_text.txt', 'w', encoding='utf-8') as file:
    file.write('\n'.join(all_texts))

Streaming output truncated to the last 5000 lines.
[**2014-06-03**] 05:27AM BLOOD calTIBC-226* Ferritn-35 TRF-174*
[**2014-06-03**] 05:27AM BLOOD Albumin-2.7* Calcium-7.6* Phos-3.9 Mg-1.8  Iron-32
[**2014-06-04**] 05:18AM BLOOD Glucose-85 UreaN-11 Creat-0.6 Na-137  K-4.1 Cl-108 HCO3-23 AnGap-10
[**2014-06-04**] 05:18AM BLOOD WBC-4.2 RBC-3.19* Hgb-9.9* Hct-27.8*  MCV-87 MCH-31.1 MCHC-35.7* RDW-12.8 Plt Ct-184

Left Foot X-Ray: No fracture or dislocation 

Brief Hospital Course:
63 yo female with multiple medical problems who presented s/p fall and found to be hypotensive secondary to dehydration and ingestion of atenolol that seemed to have resolved with IVF.

1. Hypotension:
- likely secondary to dehydration as responded well to IVF and did have some acute renal failure that resolved with fluids - could have been superimposed with atenolol overdose although patient denies taking multiple pills. BP and HR have returned to [**Location 476**] by HD#2. Will continue to hold Atenolol until 

In [ ]:
# initialize tokenizer
tokenizer = BertWordPieceTokenizer(lowercase=True)

# train tokenizer using the text file stored above
tokenizer.train(files=['combined_text.txt'], vocab_size=30_522, min_frequency=5,
                special_tokens=['<S>', '<PAD>', '</S>', '<UNK>', '<MASK>'])

In [ ]:
# create directory to store the bert model
if not os.path.exists("/content/bert_model_v1"):
  os.mkdir("/content/bert_model_v1")

# save tokenizer training using the text data
tokenizer.save_model('/content/bert_model_v1')

['/content/bert_model_v1/vocab.txt']

In [ ]:
# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = BertTokenizer.from_pretrained('/content/bert_model_v1', max_len=512)

batch = []
for text_file_path in text_file_paths:
  # open the text file and read lines
  with open(text_file_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

  # tokenize all the lines using the recently trained tokenizer (above)

  for line in lines:
    batch.append(tokenizer(line, max_length=512, padding='max_length', truncation=True))

In [ ]:
# convert inputs_ids and attention masks to torch tensors
labels = torch.tensor([x.input_ids for x in batch])
mask = torch.tensor([x.attention_mask for x in batch])

# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
# loop through each row in input_ids tensor
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # custom [MASK] token == 3

encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}


In [ ]:
dataset = Dataset(encodings)

In [ ]:
validation_split = .2

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [ ]:
# load dataset for training the model
trainloader = torch.utils.data.DataLoader(dataset, batch_size=8, sampler= train_sampler)
validloader = torch.utils.data.DataLoader(dataset, batch_size=8, sampler= valid_sampler)

In [ ]:
len(trainloader)

652

In [ ]:
len(validloader)

163

In [ ]:
config = BertConfig(
    vocab_size=30_522,
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
    position_embedding_type = 'relative_key'
)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def validate(model):
  model.eval()
  rloss = 0
  with torch.no_grad():
    for batch in validloader:
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        rloss += outputs.loss.sum().item()
  return rloss/len(validloader)

In [ ]:
# define bert model using the config defined above
model = BertForMaskedLM(config)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# and move our model over to the selected device
model.to(device)
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (distance_embedding): Embedding(1027, 64)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [ ]:
def train_model(params):
  epochs = params['epochs']
  lr = params['lr']
  wt_decay = params['wt_decay']
  amsgrad = params['ams']
  optim = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=wt_decay, amsgrad=amsgrad)
  for epoch in range(int(epochs)):
      rloss = 0
      for batch in trainloader:
          # initialize calculated gradients (from prev step)
          optim.zero_grad()
          # pull all tensor batches required for training
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)
          # process
          outputs = model(input_ids, attention_mask=attention_mask,
                          labels=labels)
          # extract loss
          loss = outputs.loss.mean()
          # calculate loss for every parameter that needs grad update
          loss.backward()
          # update parameters
          optim.step()
          rloss += outputs.loss.sum().item()
      val_loss = validate(model)
      print("Epoch {} Training loss {} Valdation Loss {}".format(epoch, rloss/len(trainloader), val_loss))
  # save trained model
  model.save_pretrained('bert_model_v1')
  return

In [ ]:
params = {
    'epochs': 5,
    'lr': 2e-5,
    'wt_decay': 0.01,
    'ams': True
}

In [ ]:
train_model(params)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Epoch 0 Training loss 0.39375088995242596 Valdation Loss 0.24387062452962063
Epoch 1 Training loss 0.21006285075235404 Valdation Loss 0.20542871206998825


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -r "/content/drive/MyDrive/bert_model_d1.zip" "bert_model_v1"

  adding: bert_model_v1/ (stored 0%)
  adding: bert_model_v1/vocab.txt (deflated 55%)
  adding: bert_model_v1/config.json (deflated 47%)
  adding: bert_model_v1/generation_config.json (deflated 8%)
  adding: bert_model_v1/model.safetensors (deflated 5%)
